* Flask for REST API Development
* Setup Flask Project for REST APIs
* Develop GET API for Users
* Add GET and DELETE API for User
* Add DELETE API for User
* Exercise and Solution - Develop APIs for Courses

* Flask for REST API Development

Flask is more popular for REST API Development. In those scenarios, the front end will be developed using Java Scrip frameworks such as Angular, React, Vue, etc.

* Setup Flask Project for REST APIs

1. Setup Project using VS Code (sales-app-rest).
2. Create Python Virtual Environment by name `sa-venv`. Make sure it is integrated with the workspace.
3. Create `requirements.txt` with required dependencies and install the dependencies.

```shell
Flask==2.3.2
```

4. Create `app.py` with the following code.

```python
from flask import Flask


app = Flask(__name__)


@app.route('/')
def hello_world():
    return {'message': 'Hello World'}
```

5. Run the application and then validate GET API for Users using below `curl` command.`

```shell
python -m flask run --debug
curl http://localhost:5000 # Run this in another terminal
```

* Develop GET API for Users

1. Add required dependencies for database connectivity.

```shell
Flask==2.3.2
Flask-SQLAlchemy==3.0.3
psycopg2-binary==2.9.6
python-dotenv==1.0.0
```

2. Setup `.env` file with database connectivity information.

```text
SALES_DB_HOST=localhost
SALES_DB_PORT=5432
SALES_DB_NAME=sales_db
SALES_DB_USER=sales_user
SALES_DB_PASS=itversity
```

3. ADD `models/user.py` with below code.

```python
from app import db


class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    first_name = db.Column(db.String)
    last_name = db.Column(db.String)
    username = db.Column(db.String, unique=True, nullable=False)
    email = db.Column(db.String)
```

4. Add `routes/user_routes.py` with below code.

```python
from flask import jsonify, request

from app import app
from app import db
from models.user import User


@app.route('/users')
def users():
    search_email = request.args.get('email', '')  

    if search_email:
        # Query the database and filter users based on the pattern match
        user_recs = db.session.query(User).filter(User.email.like(f'{search_email.lower()}%')).all()
    else:
        # Retrieve all users if no search query is provided
        user_recs = db.session.query(User).all()

    # below commented code throws error as the SQL Alchemy 
    # get response includes attributes such as _sa_instance_state. 
    # The values of these attributes are not JSON Serializable

    # users = list(map(lambda rec: rec.__dict__, user_recs))
    
    # Fix: We need to Serialize Database Model Objects to JSON
    users = []
    for user in user_recs:
        user.__dict__.pop('_sa_instance_state')
        users.append(user.__dict__)

    return jsonify(users)
```

5. Update `app.py` with the code integrating with user routes.

```python
import os
from dotenv import load_dotenv
from flask import Flask
from flask_sqlalchemy import SQLAlchemy


load_dotenv('.env')
db = SQLAlchemy()
app = Flask(__name__)
app.logger.setLevel('INFO')
host = os.environ.get('SALES_DB_HOST')
port = os.environ.get('SALES_DB_PORT')
db_name = os.environ.get('SALES_DB_NAME')
user = os.environ.get('SALES_DB_USER')
password = os.environ.get('SALES_DB_PASS')
app.config['SQLALCHEMY_DATABASE_URI'] = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'
db.init_app(app)

from routes import user_routes
from models.user import User


@app.route('/')
def hello_world():
    return 'Hello World'
```

* Add GET and DELETE API for User

1. Add `GET` and `DELETE` functionality to `routes/user_routes.py`

```python
@app.route('/user', methods=['GET', 'DELETE'])
def user():
    id = request.args.get('id')
    if request.method == 'GET':
        if id:
            user = User.query.get(id)
            user.__dict__.pop('_sa_instance_state')
            return jsonify(user.__dict__), 200
    elif request.method == 'DELETE':
        if id:
            user = User.query.get(id)
            db.session.delete(user)
            db.session.commit()
            return jsonify({'message': 'User deleted successfully...'}), 200
```

2. Launch flask shell (`python -m flask shell`) and validate `GET` as well as `DELETE`

```python
User.query.all()

from werkzeug.test import Client
client = Client(app)

# GET
client.get('/user?id=3').get_json()

# DELETE
client.get('/user?id=3')

User.query.all()
```

* Add POST for User

1. Add `INSERT` and `UPDATE` functionality using `POST` to `routes/user_routes.py`.

```python
@app.route('/user', methods=['GET', 'POST', 'DELETE'])
def user():
    id = request.args.get('id')
    if request.method == 'GET':
        if id:
            user = User.query.get(id)
            user.__dict__.pop('_sa_instance_state')
            return jsonify(user.__dict__), 200
    elif request.method == 'POST':
        id = request.form['id']
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        username = request.form['username']
        email = request.form['email']
        if id:
            user = User.query.get(id)
            user.first_name = first_name
            user.last_name = last_name
            user.username = username
            user.email = email
            db.session.commit()
            return jsonify({'message': 'User updated successfully...'}), 200
        else:
            user = User(
                first_name=first_name,
                last_name=last_name,
                username=username,
                email=email
            )
            db.session.add(user)
            db.session.commit()
            return jsonify({'message': 'User added successfully...'}), 200
    elif request.method == 'DELETE':
        if id:
            user = User.query.get(id)
            db.session.delete(user)
            db.session.commit()
            return jsonify({'message': 'User deleted successfully...'}), 200
```

2. Launch flask shell (`python -m flask shell`) and validate `POST` (insert and update).

```python
from werkzeug.test import Client
client = Client(app)

# Insert or add
form_data = {
    'id': '',
    'first_name': 'Durga',
    'last_name': 'LNF',
    'username': 'dlnf',
    'email': 'dlnf@email.com'
}
client.post('/user', data=form_data)
User.query.filter(User.email == 'dlnf@email.com').all()[0].__dict__

# Update
form_data = {
    'id': 'replace id based on filter outcome',
    'first_name': 'Durga',
    'last_name': 'Gadiraju',
    'username': 'dgadiraju',
    'email': 'dgadiraju@email.com'
}
client.post('/user', data=form_data)
User.query.filter(User.email == 'dgadiraju@email.com').all()[0].__dict__
```

* Exercise - Develop APIs for Courses

1. Create Model for Course
2. Add Route to list the existing courses
3. Add Route for GET, POST, as well as DELETE
4. Make sure `app.py` is updated to use the route
5. Validate all the APIs using Flask Shell 

* Exercise - Develop APIs for Courses

1. Create Model for Course (`models/course.py`)

```python
from app import db


class Course(db.Model):
    __tablename__ = 'courses'
    course_id = db.Column(db.Integer, primary_key=True)
    course_name = db.Column(db.String)
    course_author = db.Column(db.String)
    course_endpoint = db.Column(db.String)
```

2. Add Route to list the existing courses

```python
from flask import jsonify, request

from app import app
from app import db
from models.course import Course


@app.route('/courses')
def courses():
    search_query = request.args.get('search', '')

    if search_query:
        # Query the database for courses matching the search query
        course_recs = db.session. \
            query(Course). \
            filter(Course.course_name.ilike(f"%{search_query}%")). \
            all()
    else:
        # Query the database for all courses
        course_recs = db.session.query(Course).all()

    courses = []
    for course in course_recs:
        course.__dict__.pop('_sa_instance_state')
        courses.append(course.__dict__)
    return jsonify(courses)
```

3. Add Route for GET, POST, as well as DELETE

```python
@app.route('/course', methods=['GET', 'POST', 'DELETE'])
def course():
    course_id = request.args.get('course_id')
    if request.method == 'GET':
        if course_id:
            course = Course.query.get(course_id)
            course.__dict__.pop('_sa_instance_state')
            return jsonify(course.__dict__), 200
    elif request.method == 'POST':
        course_id = request.form['course_id']
        course_name = request.form['course_name']
        course_author = request.form['course_author']
        course_endpoint = request.form['course_endpoint']
        if course_id:
            course = Course.query.get(course_id)
            course.course_name = course_name
            course.course_author = course_author
            course.course_endpoint = course_endpoint
            db.session.commit()
            return jsonify({'message': 'Course updated successfully...'}), 200
        else:
            course = Course(
                course_name=course_name,
                course_author=course_author,
                course_endpoint=course_endpoint,
            )
            db.session.add(course)
            db.session.commit()
            return jsonify({'message': 'Course added successfully...'}), 200
    elif request.method == 'DELETE':
        if course_id:
            course = Course.query.get(course_id)
            db.session.delete(course)
            db.session.commit()
            return jsonify({'message': 'Course deleted successfully...'}), 200
```

4. Make sure `app.py` is updated to use the route

```python
import os
from dotenv import load_dotenv
from flask import Flask
from flask_sqlalchemy import SQLAlchemy


load_dotenv('.env')
db = SQLAlchemy()
app = Flask(__name__)
app.logger.setLevel('INFO')
host = os.environ.get('SALES_DB_HOST')
port = os.environ.get('SALES_DB_PORT')
db_name = os.environ.get('SALES_DB_NAME')
user = os.environ.get('SALES_DB_USER')
password = os.environ.get('SALES_DB_PASS')
app.config['SQLALCHEMY_DATABASE_URI'] = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'
db.init_app(app)

from routes import user_routes, course_routes
from models.user import User
from models.course import Course


@app.route('/')
def hello_world():
    return 'Hello World'
```

5. Validate all the APIs using Flask Shell

```python
from werkzeug.test import Client
client = Client(app)

client.get('/courses').get_json()
client.get('/course?course_id=1').get_json()

# Insert
form_data = {
    'course_id': '',
    'course_name': 'Python Essentials',
    'course_author': 'Durga',
    'course_endpoint': 'python-essentials'
}

client.post('/course', data=form_data)

Course.query.filter(Course.course_name.like('Python%')).all()[0].__dict__

# Update
form_data = {
    'course_id': input('Enter Course id to be updated: '),
    'course_name': 'Python Essentials',
    'course_author': 'Durga',
    'course_endpoint': 'python-essentials'
}

client.post('/course', data=form_data)

Course.query.filter(Course.course_name.like('Python%')).all()[0].__dict__

# Delete
client.delete('/course?course_id=<add course id>')
```